### Department of Management

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np

In [2]:
r_mgt=requests.get('https://www.lse.ac.uk/management/people-home')
soup_mgt=BeautifulSoup(r_mgt.content,'lxml')

In [3]:
mgt=[]

label1=soup_mgt.find_all('h1')
label1_wanted1=['academic staff',
               'other academic and research staff']
label1_wanted2=['professional services staff']

for label1 in soup_mgt.find_all('h1'):
    department='management'
    
    if label1.get_text().lower() in label1_wanted1:
        shortcut=label1.find_next('div',attrs={'class':'accordionContainer'})
        for label2 in shortcut.find_all('h2',attrs={'class':'accordion__title'}):
            inlabel2=label2.find_next('div',attrs={'class':'accordion__content'})
            label=label2.get_text()
            for person in inlabel2.find_all('div',attrs={'class':'accordion__txt'}):
                if person.find('p').find('strong'):
                    name=person.find('p').find('strong').get_text()
                    mgt.append([name,department,label]) 
                else:
                    name=person.find('p').find_next('a').get_text()
                    mgt.append([name,department,label])
    
    if label1.get_text().lower() in label1_wanted2:
        shortcut=label1.find_next('div',attrs={'class':'accordionContainer'})
        for label2 in shortcut.find_all('h2',attrs={'class':'accordion__title'}):
            inlabel2=label2.find_next('div',attrs={'class':'accordion__content'})
            label=label2.get_text()
            for person in inlabel2.find_all('div',attrs={'class':'accordion__txt'}):
                if person.find('p').find('strong'):
                    name=person.find('p').find('strong').get_text()
                    mgt.append([name,department,label]) 
                elif person.find('p').find('b'):
                    name=person.find('p').find('b').get_text()
                    mgt.append([name,department,label]) 
                elif person.find('p').find('span'):
                    name=person.find('p').find('span').get_text()
                    mgt.append([name,department,label]) 
                else:
                    print('not all included')
    

In [4]:
for i in range(len(mgt)):
    namestr=mgt[i][0].lower().split()
    if 'dr' in namestr:
        mgt[i].append('Dr')
    elif 'professor' in namestr:
        mgt[i].append('Professor')
    else:
        mgt[i].append('Non')

    name=mgt[i][0].replace('Dr','').replace('Professor','').replace('Sir','').replace('\xa0',' ').split()
    name=" ".join(name)
    mgt[i][0]=name

In [5]:
mgt=pd.DataFrame(mgt,columns=['name','department','label','title'])
mgt

,name,department,label,title
0,Bethania Antunes,management,Employment Relations and Human Resource Manage...,Dr
1,Sarah Ashwin,management,Employment Relations and Human Resource Manage...,Professor
2,Jonathan E. Booth,management,Employment Relations and Human Resource Manage...,Dr
3,Wafaa Elmezraoui,management,Employment Relations and Human Resource Manage...,Non
4,Karin King,management,Employment Relations and Human Resource Manage...,Dr
...,...,...,...,...
168,Paul Willman,management,Visiting Staff and Emeriti,Professor
169,Mohamed Abouaziza,management,Research staff,Dr
170,Anushri Gupta,management,Research staff,Dr
171,Philipp Schoenegger,management,Research staff,Dr


In [6]:
print('\nSome names called vacancies are accidentally included, as they directly appear on the webpage.')
display(mgt[mgt['name']=='Vacancy'])
print('\n\nThere is one name missing because of the abnormal structure of the webpage')
display(mgt[mgt['name']==''])
display(mgt[148:151])


Some names called vacancies are accidentally included, as they directly appear on the webpage.


,name,department,label,title
75,Vacancy,management,Taught Programmes,Non
82,Vacancy,management,Taught Programmes,Non
100,Vacancy,management,Executive Programmes,Non
122,Vacancy,management,"Marketing, Analytics and Communications",Non
133,Vacancy,management,Faculty Affairs,Non




There is one name missing because of the abnormal structure of the webpage


,name,department,label,title
149,,management,Visiting Staff and Emeriti,Dr


,name,department,label,title
148,Michele Fioretti,management,Visiting Staff and Emeriti,Dr
149,,management,Visiting Staff and Emeriti,Dr
150,Dina Rabie,management,Visiting Staff and Emeriti,Dr


In [7]:
mgt['name'] = mgt['name'].replace('Vacancy', pd.NA)
mgt=mgt.copy().dropna()
mgt.loc[mgt['name']=='','name']='Dr Henry Hang Shen'

display(mgt[mgt['name']=='Vacancy'])
display(mgt[mgt['name']==''])

,name,department,label,title


,name,department,label,title


In [8]:
mgt

,name,department,label,title
0,Bethania Antunes,management,Employment Relations and Human Resource Manage...,Dr
1,Sarah Ashwin,management,Employment Relations and Human Resource Manage...,Professor
2,Jonathan E. Booth,management,Employment Relations and Human Resource Manage...,Dr
3,Wafaa Elmezraoui,management,Employment Relations and Human Resource Manage...,Non
4,Karin King,management,Employment Relations and Human Resource Manage...,Dr
...,...,...,...,...
168,Paul Willman,management,Visiting Staff and Emeriti,Professor
169,Mohamed Abouaziza,management,Research staff,Dr
170,Anushri Gupta,management,Research staff,Dr
171,Philipp Schoenegger,management,Research staff,Dr


In [9]:
len(mgt)

168

In [10]:
#72+36=108
#108+65=173

### Department of Government

In [11]:
r_gvt=requests.get('https://www.lse.ac.uk/government/people')
soup_gvt=BeautifulSoup(r_gvt.content,'lxml')

In [12]:
gvt=[]

label1=soup_gvt.find_all('h2',attrs={'class':'accordion__title'})
llabel1_wanted1=['academic staff',
               'professional services staff',
                'research staff']
label1_wanted2=['guest teachers and gtas']
label1_wanted3=['emeritus, affiliated & visiting academic staff']

for label1 in soup_gvt.find_all('h2',attrs={'class':'accordion__title'}):
    department='government'
    if label1.get_text().lower() in llabel1_wanted1:
        label=label1.get_text()
        shortcut=label1.find_next('div',attrs={'class':'accordion__content'})
        for person in shortcut.find_all('div',attrs={'class':'accordion__txt'}):
            if person.find('p').find('strong'):
                name=person.find('p').find('strong').get_text()
                gvt.append([name,department,label])
            elif person.find('p').find('b'):
                name=person.find('p').find('b').get_text()
                gvt.append([name,department,label])
            else:
                print('not all structure considered')
    
    if label1.get_text().lower() in label1_wanted2:
        label=label1.get_text()
        shortcut=label1.find_next('div',attrs={'class':'accordion__content'})
        for person in shortcut.find_all('p'):
            if person.find('strong'):
                continue
            else:
                name=person.get_text()
                gvt.append([name,department,label])

    if label1.get_text().lower() in label1_wanted3:
        label=label1.get_text()
        shortcut=label1.find_next('div',attrs={'class':'accordion__content'})
        for people in shortcut.find_all('ul'):
            for person in people.find_all('li'):
                if person.find('p'):
                    if person.find('p').find('strong'):
                        name=person.find('p').find('strong').get_text()
                        gvt.append([name,department,label])
                    else:
                        name=person.find('p').find('span').get_text()
                        gvt.append([name,department,label])
                else:
                    if person.find('strong'):
                        name=person.find('strong').get_text()
                        gvt.append([name,department,label])
                    else:
                        print('Not all structures considered')
    
    

In [13]:
for i in range(len(gvt)):
    namestr=gvt[i][0].lower().split()
    if '(dr)' in namestr:
        gvt[i].append('Dr')
    elif '(prof)' in namestr:
        gvt[i].append('Professor')
    else:
        gvt[i].append('Non')

    namestrings=[]
    name=gvt[i][0].replace('\xa0',' ').split()
    for stri in name:
        string=stri.strip('()')
        if string.startswith('GV'):
            continue
        elif string in ['Dr','Mr','Prof']:
            continue
        else:
            namestrings.append(stri)
        
    name=" ".join(namestrings)
    gvt[i][0]=name

In [14]:
gvt=pd.DataFrame(gvt,columns=['name','department','label','title'])
gvt

,name,department,label,title
0,Victor Agboga,government,Academic Staff,Non
1,Elise Antoine,government,Academic Staff,Dr
2,Paul Apostolidis,government,Academic Staff,Professor
3,Tom Bailey,government,Academic Staff,Non
4,Daniel Berliner,government,Academic Staff,Dr
...,...,...,...,...
158,Lukas Slothuus,government,"Emeritus, Affiliated & Visiting Academic Staff",Dr
159,Zeynep Somer Topcu,government,"Emeritus, Affiliated & Visiting Academic Staff",Dr
160,Christine Stedtnitz,government,"Emeritus, Affiliated & Visiting Academic Staff",Dr
161,Jill Stuart,government,"Emeritus, Affiliated & Visiting Academic Staff",Dr


In [15]:
len(gvt)

163

In [16]:
55+14+2+39+53

163

### LSE Law School

In [17]:
r_law=requests.get('https://www.lse.ac.uk/law/people')
soup_law=BeautifulSoup(r_law.content,'lxml')

In [18]:
law=[]

#label1=soup_law.find_all('h2',attrs={'class':'accordion__title'})
label1_wanted1=['A > H','I > O','P > Z','LSE Fellows']
label1_wanted2=['Professional services staff']
label1_wanted3=['Emeritus']

for label1 in soup_law.find_all('h2',attrs={'class':'accordion__title'}):
    department='law'
    if any(text in label1.get_text() for text in label1_wanted1):
        label=label1.get_text().replace('\xa0',' ')
        shortcut=label1.find_next('div',attrs={'class':'accordion__content'})
        for person in shortcut.find_all('div',attrs={'class':'accordion__txt'}):
            name=person.find('a')['title'].replace('\xa0',' ')
            law.append([name,department,label])
            
    if label1.get_text() in label1_wanted2:
        label=label1.get_text().replace('\xa0',' ')
        shortcut=label1.find_next('div',attrs={'class':'accordion__content'})
        for person in shortcut.find_all('div',attrs={'class':'accordion__txt'}):
            if person.find('p').find('a'):
                name=person.find('p').find('a').get_text().replace('\xa0',' ')
                law.append([name,department,label])
            elif person.find('p').find('strong'):
                name=person.find('p').find('strong').get_text().replace('\xa0',' ')
                law.append([name,department,label])
            else:
                print('Not all structures considered')
    
    if any(text in label1.get_text() for text in label1_wanted3):
        label=label1.get_text().replace('\xa0',' ')
        shortcut=label1.find_next('div',attrs={'class':'accordion__content'}).find_next('ul')
        for person in shortcut.find_all('li'):
            if person.find('p').find('a'):
                name=person.find('p').find('a')['title']
                law.append([name,department,label])
            else:
                name=person.find('p').get_text()
                law.append([name,department,label])


In [19]:
25+21+27+5+25+13

116

In [20]:
len(law)

116

In [21]:
'home'

'home'

In [22]:
law

[['Cressida Auckland', 'law', 'Academic staff by surname: A > H'],
 ['Professor Susanne Baer', 'law', 'Academic staff by surname: A > H'],
 ['Eduardo Baistrocchi', 'law', 'Academic staff by surname: A > H'],
 ['Chaloka Beyani', 'law', 'Academic staff by surname: A > H'],
 ['Julia Black', 'law', 'Academic staff by surname: A > H'],
 ['Michael Blackwell', 'law', 'Academic staff by surname: A > H'],
 ['Jacco Bomhoff', 'law', 'Academic staff by surname: A > H'],
 ['Jo Braithwaite', 'law', 'Academic staff by surname: A > H'],
 ['Nafay Choudhury', 'law', 'Academic staff by surname: A > H'],
 ['Hugh Collins', 'law', 'Academic staff by surname: A > H'],
 ['Sir Ross Cranston', 'law', 'Academic staff by surname: A > H'],
 ['Niamh Dunne', 'law', 'Academic staff by surname: A > H'],
 ['Neil Duxbury', 'law', 'Academic staff by surname: A > H'],
 ['Tatiana Flessas', 'law', 'Academic staff by surname: A > H'],
 ['Conor Gearty', 'law', 'Academic staff by surname: A > H'],
 ['Hannah Gibbs', 'law', 'Aca

In [23]:
law=[]

label1=soup_law.find_all('h2',attrs={'class':'accordion__title'})
for label in label1:
    print(label.get_text())
  

LSE Law School Leadership Team
Academic staff by surname: A > H
Academic staff by surname: I > O
Academic staff by surname: P > Z
Academic staff (Emeritus) 
LSE Fellows
Professional services staff
Senior Fellows
Visiting Professors & Visiting Professors in Practice
Visiting Fellows
Visiting Fellows in Practice


name
Jill Stuart          2
Geoffrey Myers       2
Victor Agboga        1
Sebastian Balfour    1
Sarah Tustin         1
Name: count, dtype: int64

name
Bethania Antunes    1
Paddy Hyde          1
Bethan Briggs       1
Ardy Cheung         1
Molly Healy         1
Name: count, dtype: int64